In [1]:
from struct import unpack
from pathlib import Path
from enum import Enum

import json
import xmltodict
import numpy as np

import matplotlib.pyplot as plt

from astropy.time import Time
import astropy.units as u

from timdimm_tng.scheduler import Sequence, Observation, Schedule

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [8]:
s = Sequence(template="/Users/tim/timdimm_test.esq")
s

{'SequenceQueue': {'@version': '2.5', 'Observer': 'timDIMM', 'GuideDeviation': {'@enabled': 'false', '#text': '2'}, 'GuideStartDeviation': {'@enabled': 'false', '#text': '2'}, 'Autofocus': {'@enabled': 'false', '#text': '0'}, 'RefocusOnTemperatureDelta': {'@enabled': 'false', '#text': '1'}, 'RefocusEveryN': {'@enabled': 'false', '#text': '60'}, 'RefocusOnMeridianFlip': {'@enabled': 'false'}, 'Job': {'Exposure': '0.001', 'Format': 'Mono', 'Encoding': 'FITS', 'Binning': {'X': '1', 'Y': '1'}, 'Frame': {'X': '0', 'Y': '0', 'W': '1608', 'H': '1104'}, 'Temperature': {'@force': 'false', '#text': '0'}, 'Filter': None, 'Type': 'Light', 'Prefix': {'FilterEnabled': '0', 'ExpEnabled': '0', 'TimeStampEnabled': '0'}, 'Count': '1', 'Delay': '0', 'PreCaptureScript': 'timdimm_precapture', 'PostCaptureScript': 'timdimm_postcapture', 'PreJobScript': 'timdimm_prejob', 'PostJobScript': 'timdimm_postjob', 'FITSDirectory': './', 'PlaceholderFormat': '/%t/%T/%F/%t_%T_%F', 'PlaceholderSuffix': '3', 'UploadMode

In [9]:
Observation()

{'Name': 'Target', 'Priority': '10', 'Coordinates': {'J2000RA': '0.0', 'J2000DE': '0.0'}, 'PositionAngle': '', 'Sequence': '/Users/tim/sequence.esq', 'StartupCondition': {'Condition': 'ASAP'}, 'Constraints': {'Constraint': [{'@value': '45', '#text': 'MinimumAltitude'}, 'EnforceTwilight', 'EnforceArtificialHorizon']}, 'CompletionCondition': {'Condition': {'@value': '100', '#text': 'Repeat'}}, 'Steps': {'Step': 'Track'}, 'Target': 'Target'}

In [10]:
Schedule()

{'SchedulerList': {'@version': '1.5', 'Profile': 'Default', 'Job': [], 'SchedulerAlgorithm': {'@value': '0'}, 'ErrorHandlingStrategy': {'@value': '1', 'delay': '0'}, 'StartupProcedure': None, 'ShutdownProcedure': None}}